In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import time
import sys
import rtsvg
rt = rtsvg.RACETrack()

# mostly a duplicate of the circle_routing...

_n_paths_         = 20
_n_circles_       = 100
_radius_min_      = 20
_radius_max_      = 30
_circle_geoms_    = []
_min_circle_sep_  = 30
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 4
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 10                     # less than the _min_circle_sep_

def circleOverlaps(cx, cy, r):
    for _geom_ in _circle_geoms_:
        dx, dy = _geom_[0] - cx, _geom_[1] - cy
        d      = sqrt(dx*dx+dy*dy)
        if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
            return True
    return False

def findOpening():
    _max_attempts_ = 100
    attempts  = 0
    cx, cy, r = random.randint(_radius_max_+_min_circle_sep_, 600-_radius_max_-_min_circle_sep_),     random.randint(_radius_max_+_min_circle_sep_, 400-_radius_max_-_min_circle_sep_), random.randint(_radius_min_,_radius_max_)
    while circleOverlaps(cx,cy,r) and attempts < _max_attempts_:
        cx, cy, r = random.randint(_radius_max_+_min_circle_sep_, 600-_radius_max_-_min_circle_sep_), random.randint(_radius_max_+_min_circle_sep_, 400-_radius_max_-_min_circle_sep_), random.randint(_radius_min_,_radius_max_)
        attempts += 1
    if attempts == _max_attempts_:
        return None
    return cx, cy, r

for i in range(_n_circles_):
    to_unpack = findOpening()
    if to_unpack is not None:
        cx, cy, r = to_unpack
        _circle_geoms_.append((cx,cy,r))

# points to connect
_pts_ = []
c0    = random.randint(0, len(_circle_geoms_)-1)
c1    =  random.randint(0, len(_circle_geoms_)-1)
while c1 == c0:
    c1 =  random.randint(0, len(_circle_geoms_)-1)
a0, a1 = random.random() * 2 * pi, random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c0]
_pts_.append((cx+(r+_radius_start_+1)*cos(a0), cy+(r+_radius_start_+1)*sin(a0)))
_pts_.append((cx+(r+_escape_px_)*cos(a0), cy+(r+_escape_px_)*sin(a0)))
cx, cy, r = _circle_geoms_[c1]
_pts_.append((cx+(r+_escape_px_)*cos(a1), cy+(r+_escape_px_)*sin(a1)))
_pts_.append((cx+(r+_radius_start_+1)*cos(a1), cy+(r+_radius_start_+1)*sin(a1)))

entries         = [_pts_]
_exit_segments_ = []
_exit_segments_.append(_pts_[2:])
for i in range(_n_paths_):
    _more_pts_ = [_pts_[0], _pts_[1]]
    c          = random.randint(0,len(_circle_geoms_)-1)
    while c == c0:
        c  = random.randint(0,len(_circle_geoms_)-1)
    a = random.random() * 2 * pi
    cx, cy, r = _circle_geoms_[c]
    _more_pts_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
    _more_pts_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))
    _exit_segments_.append(_more_pts_[2:])
    entries.append(_more_pts_)

In [ ]:
def calculatePathAroundCircles(pts, circle_geoms, radius_inc_test, half_sep):
    def breakSegment(_segment_):
        if rt.segmentLength(_segment_) < 2.0:
            return _segment_
        for _geom_ in circle_geoms:
            _circle_plus_ = (_geom_[0], _geom_[1], _geom_[2]+radius_inc_test)
            _dist_, _inter_  = rt.segmentIntersectsCircle(_segment_,_circle_plus_)
            if _dist_ <= _circle_plus_[2]:
                if _inter_[0] == _geom_[0] and _inter_[1] == _geom_[1]:
                    dx, dy   = _segment_[1][0] - _segment_[0][0], _segment_[1][1] - _segment_[0][1]
                    l        = sqrt(dx*dx+dy*dy)
                    dx,  dy  = dx/l, dy/l
                    pdx, pdy = -dy, dx 
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + pdx*(_geom_[2]+half_sep), _geom_[1] + pdy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
                else:
                    dx, dy = _inter_[0] - _geom_[0], _inter_[1] - _geom_[1]
                    l      = sqrt(dx*dx+dy*dy)
                    dx, dy = dx/l, dy/l
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0]  + dx*(_geom_[2]+half_sep), _geom_[1]  + dy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
        return _segment_

    last_length = 0
    _segments_  = []
    for _pt_ in pts:
        _segments_.append(_pt_)
    while last_length != len(_segments_):
        last_length    = len(_segments_)
        _new_segments_ = []
        for i in range(len(_segments_)-1):
            _new_ = breakSegment([_segments_[i], _segments_[i+1]])
            if len(_new_) == 3:
                _new_segments_.append(_new_[0])
                _new_segments_.append(_new_[1])
            else:
                _new_segments_.append(_new_[0])
        _new_segments_.append(_new_[-1])
        _segments_ = _new_segments_
        
    return _segments_

all_paths = []
for x in entries:
    _segments_ = calculatePathAroundCircles(x, _circle_geoms_, _radius_inc_test_, _half_sep_)
    all_paths.append(_segments_)

svg = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />'
# Render Paths
for _path_ in all_paths:
    for i in range(len(_path_)-1):
        svg += f'<line x1="{_path_[i][0]}" y1="{_path_[i][1]}" x2="{_path_[i+1][0]}" y2="{_path_[i+1][1]}" stroke="#000000" stroke-width="0.2" />'
# Render Entry Points
svg += f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />'
svg += f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="3" stroke="#00af00" fill="#00af00" />'
for _entry_ in entries:
    svg += f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" />'
    svg += f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="3" stroke="#ff0000" fill="#ff0000" />'
svg += '</svg>'

svg_smooth = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg_smooth += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />'
# Render Paths
for _path_ in all_paths:
    _smoothed_path_ = rt.smoothSegments(rt.expandSegmentsIntoPiecewiseCurvedParts(_path_))
    for i in range(len(_smoothed_path_)-1):
        svg_smooth += f'<line x1="{_smoothed_path_[i][0]}" y1="{_smoothed_path_[i][1]}" x2="{_smoothed_path_[i+1][0]}" y2="{_smoothed_path_[i+1][1]}" stroke="#000000" stroke-width="0.2" />'
# Render Entry Points
svg_smooth += f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />'
svg_smooth += f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="3" stroke="#00af00" fill="#00af00" />'
for _entry_ in entries:
    svg_smooth += f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" />'
    svg_smooth += f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="3" stroke="#ff0000" fill="#ff0000" />'
svg_smooth += '</svg>'

rt.tile([svg,svg_smooth])

In [ ]:
x_min,y_min,x_max,y_max = all_paths[0][0][0], all_paths[0][0][1], all_paths[0][0][0], all_paths[0][0][1] 
pts_total               = 0
smoothed_lu             = {}
pt_struct_lu            = {}
for i in range(len(all_paths)):
    _path_ = all_paths[i]
    _smoothed_path_ = rt.smoothSegments(rt.expandSegmentsIntoPiecewiseCurvedParts(_path_))
    smoothed_lu[tuple(_path_)] = _smoothed_path_
    for j in range(len(_smoothed_path_)):
        _pt_ = _smoothed_path_[j]
        pts_total += 1
        x_min, x_max = min(x_min, _pt_[0]), max(x_max, _pt_[0])
        y_min, y_max = min(y_min, _pt_[1]), max(y_max, _pt_[1])
        if _pt_ not in pt_struct_lu.keys():
            pt_struct_lu[_pt_] = set()
        pt_struct_lu[_pt_].add((i,j))

In [ ]:
my_xy_tree = rt.xyQuadTree((x_min,y_min,x_max,y_max), max_pts_per_node=50)

svg2 = []
svg2.append('<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />')
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg2.append(f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />')

# Sort the paths by distance
sorter = []
for i in range(len(entries)):
    d = rt.segmentLength(((entries[i][ 0][0],entries[i][ 0][1]),(entries[i][-1][0],entries[i][-1][1])))
    sorter.append((d,i))
sorter = sorted(sorter)

# Render Longest Path First
pts_plotted = 0
_path_ = smoothed_lu[tuple(all_paths[sorter[-1][1]])]
_path_.reverse()
my_xy_tree.add(_path_) # Add the longest path in first
for i in range(len(_path_)-1):
    svg2.append(f'<line x1="{_path_[i][0]}" y1="{_path_[i][1]}" x2="{_path_[i+1][0]}" y2="{_path_[i+1][1]}" stroke="#000000" stroke-width="0.2" />')
    pts_plotted += 1

# Now all subsequent paths ... attempting to find merge points...
for i in range(len(all_paths)-2, -1, -1):
    _path_ = smoothed_lu[tuple(all_paths[sorter[i][1]])]
    # Add the exit stem...
    svg2.append(f'<line x1="{_path_[-1][0]}" y1="{_path_[-1][1]}" x2="{_path_[-2][0]}" y2="{_path_[-2][1]}" stroke="#000000" stroke-width="0.2" />')
    # Then start adding pieces until we find a merge point
    for j in range(len(_path_)-2, 0, -1):
        e0,e1 = _path_[j],_path_[j-1]

        # Is there a merge point?
        closests  = my_xy_tree.closest(e1,n=1)
        matching  = None
        if closests[0][0] < 2.0:
            uv = rt.unitVector((e0,e1))
            possible_matching = (closests[0][1][0],closests[0][1][1])
            for x in pt_struct_lu[possible_matching]:
                path_i, point_j = x[0], x[1]
                smoothed_matching = smoothed_lu[tuple(all_paths[path_i])]
                if point_j > 0:
                    next_matching = smoothed_matching[point_j-1]
                    uv_possible = rt.unitVector((possible_matching,next_matching))
                    _dot_       = uv[0]*uv_possible[0] + uv[1]*uv_possible[1]
                    if _dot_ > 0.99:
                        matching = possible_matching
                        break

        if matching is None:
            svg2.append(f'<line x1="{e0[0]}" y1="{e0[1]}" x2="{e1[0]}" y2="{e1[1]}" stroke="#000000" stroke-width="0.2" />')
            pts_plotted +=1
        else:
            svg2.append(f'<line x1="{e0[0]}" y1="{e0[1]}" x2="{matching[0]}" y2="{matching[1]}" stroke="#ff0000" stroke-width="1.0" />')
            # svg2.append(f'<circle cx="{matching[0]}" cy="{matching[1]}" r="10" fill="none" stroke="#ff0000" stroke-width="1.0" />')
            pts_plotted +=1
            break
    _path_ = _path_[j:]
    _path_.reverse()
    my_xy_tree.add(_path_)

# Render Entry Points
svg2.append(f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />')
svg2.append(f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="3" stroke="#00af00" fill="#00af00" />')
for _entry_ in entries:
    pass
    #svg2.append(f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" />')
    #svg2.append(f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="3" stroke="#ff0000" fill="#ff0000" />')
    svg2.append(f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="2" stroke="#a0a0a0" fill="#404040" />')
svg2.append('</svg>')

print(f'total_points = {pts_total} | points_plotted = {pts_plotted}')

rt.tile([svg, ''.join(svg2)])

In [ ]:
my_xy_tree = rt.xyQuadTree((x_min,y_min,x_max,y_max), max_pts_per_node=50)

svg3 = []
svg3.append('<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />')
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg3.append(f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />')

# Sort the paths by distance
sorter = []
for i in range(len(entries)):
    d = rt.segmentLength(((entries[i][ 0][0],entries[i][ 0][1]),(entries[i][-1][0],entries[i][-1][1])))
    sorter.append((d,i))
sorter = sorted(sorter)

# Render Longest Path First
pts3_plotted = 0
_path_ = smoothed_lu[tuple(all_paths[sorter[-1][1]])]
_path_.reverse()
my_xy_tree.add(_path_) # Add the longest path in first
for i in range(len(_path_)-1):
    svg3.append(f'<line x1="{_path_[i][0]}" y1="{_path_[i][1]}" x2="{_path_[i+1][0]}" y2="{_path_[i+1][1]}" stroke="#000000" stroke-width="0.2" />')
    pts3_plotted += 1

# Now all subsequent paths ... attempting to find merge points...
for i in range(len(all_paths)-2, -1, -1):
    _path_ = smoothed_lu[tuple(all_paths[sorter[i][1]])]
    # Add the exit stem...
    svg3.append(f'<line x1="{_path_[-1][0]}" y1="{_path_[-1][1]}" x2="{_path_[-2][0]}" y2="{_path_[-2][1]}" stroke="#000000" stroke-width="0.2" />')
    # Then start adding pieces until we find a merge point
    for j in range(len(_path_)-2, 0, -1):
        e0,e1 = _path_[j],_path_[j-1]

        # Is there a merge point?
        uv = rt.unitVector((e0,e1))
        ext = ( (e1[0] - e0[0])+e1[0] , (e1[1] - e0[1])+e1[1] )
        closests  = my_xy_tree.closest(ext,n=1) # my_xy_tree.closest(e1,n=1)
        matching  = None
        if closests[0][0] < 4.0:
            uv = rt.unitVector((e0,e1))
            possible_matching = (closests[0][1][0],closests[0][1][1])            
            for x in pt_struct_lu[possible_matching]:
                path_i, point_j = x[0], x[1]
                smoothed_matching = smoothed_lu[tuple(all_paths[path_i])]
                matching = possible_matching
                break
        if matching is None:
            svg3.append(f'<line x1="{e0[0]}" y1="{e0[1]}" x2="{e1[0]}" y2="{e1[1]}" stroke="#000000" stroke-width="0.2" />')
            pts3_plotted +=1
        else:
            svg3.append(f'<line x1="{e0[0]}" y1="{e0[1]}" x2="{matching[0]}" y2="{matching[1]}" stroke="#000000" stroke-width="0.2" />')
            pts3_plotted +=1
            break
    _path_ = _path_[j:]
    _path_.reverse()
    my_xy_tree.add(_path_)

# Render Entry Points
svg3.append(f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />')
svg3.append(f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="3" stroke="#00af00" fill="#00af00" />')
for _entry_ in entries:
    pass
    #svg2.append(f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" />')
    #svg2.append(f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="3" stroke="#ff0000" fill="#ff0000" />')
    svg3.append(f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="2" stroke="#a0a0a0" fill="#404040" />')
svg3.append('</svg>')

print(f'total_points = {pts_total} | points_plotted = {pts_plotted} | points3_plotted = {pts3_plotted}')

rt.tile([''.join(svg2),''.join(svg3)])